# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 0.7821438254370431
 1.6891608719678775
 1.2097272963479444

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lu`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0         0.0        0.0
 0.0605581   1.0        0.0
 0.445687   -0.0500638  1.0
U factor:
3×3 Matrix{Float64}:
 0.936884  0.0801929   0.672084
 0.0       0.69539    -0.0155384
 0.0       0.0         0.490926

In [3]:
typeof(Alu)

LU{Float64, Matrix{Float64}, Vector{Int64}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Matrix{Float64}:
 0.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0

In [5]:
Alu.L

3×3 Matrix{Float64}:
 1.0         0.0        0.0
 0.0605581   1.0        0.0
 0.445687   -0.0500638  1.0

In [6]:
Alu.U

3×3 Matrix{Float64}:
 0.936884  0.0801929   0.672084
 0.0       0.69539    -0.0155384
 0.0       0.0         0.490926

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Vector{Float64}:
 1.0
 1.0
 1.0

In [8]:
Alu\b

3-element Vector{Float64}:
 1.0
 1.0
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qr`. 

In [10]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.0552287   0.997431    0.0456285
 -0.911996   -0.0317895  -0.408966
 -0.406465   -0.0641997   0.911408
R factor:
3×3 Matrix{Float64}:
 -1.02729  -0.112186  -0.935939
  0.0       0.695838  -0.0470657
  0.0       0.0        0.447434

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [ ]:
Aqr.Q

In [ ]:
Aqr.R

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [11]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -1.5088857315948254
  0.8882531011908853
  2.476976085071815
vectors:
3×3 Matrix{Float64}:
  0.701912   -0.501614  -0.505671
 -0.710207   -0.438977  -0.550369
  0.0540948   0.745442  -0.664372

The values and the vectors can be extracted from the Eigen type by special indexing

In [12]:
AsymEig.values

3-element Vector{Float64}:
 -1.5088857315948254
  0.8882531011908853
  2.476976085071815

In [13]:
AsymEig.vectors

3×3 Matrix{Float64}:
  0.701912   -0.501614  -0.505671
 -0.710207   -0.438977  -0.550369
  0.0540948   0.745442  -0.664372

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [14]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
 1.0          -5.27356e-16  5.55112e-16
 0.0           1.0          5.55112e-17
 1.11022e-16   6.66134e-16  1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [15]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [16]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [17]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

0.7675821990017264

In [18]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [19]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [20]:
@time eigvals(Asym);

  0.549816 seconds (268.63 k allocations: 25.897 MiB, 2.45% gc time, 64.78% compilation time)


In [21]:
@time eigvals(Asym_noisy);

  0.747013 seconds (14 allocations: 7.928 MiB)


In [22]:
@time eigvals(Asym_explicit);

  0.273668 seconds (68.34 k allocations: 12.646 MiB, 25.52% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [24]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.496503 seconds (34 allocations: 183.106 MiB, 4.16% gc time)


6.655741131690402

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [28]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [29]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 1//2   1//2  9//10
 4//5   1//1  1//5
 3//10  1//5  1//1

In [30]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 19//10
  2//1
  3//2

In [31]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [32]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}, Vector{Int64}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1  0//1  0//1
 3//8  1//1  0//1
 5//8  5//7  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 4//5   1//1    1//5
 0//1  -7//40  37//40
 0//1   0//1    4//35

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
Assign the result to variable `A_eigv`

In [35]:
using LinearAlgebra
A = [
    140 97 74 168 131
    97 106 89 131 36
    74 89 152 144 71
    168 131 144 54 142
    131 36 71 142 36
]
A_eigv = eigen(A).values

5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553057
   42.752167279318854
   87.16111477514494
  542.4677301466137

In [36]:
# Close enough
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

AssertionError: AssertionError: A_eigv == [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A` and name it `A_diag`.

In [37]:
A_diag = Diagonal(A_eigv)

5×5 Diagonal{Float64, Vector{Float64}}:
 -128.493     ⋅        ⋅        ⋅         ⋅ 
     ⋅     -55.8878    ⋅        ⋅         ⋅ 
     ⋅        ⋅      42.7522    ⋅         ⋅ 
     ⋅        ⋅        ⋅      87.1611     ⋅ 
     ⋅        ⋅        ⋅        ⋅      542.468

In [38]:
@assert isapprox(A_diag, [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468], atol=1e-3)

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [39]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [40]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]

### Please let us know how we're doing!
https://tinyurl.com/introJuliaFeedback

Please click on `Validate` on the top, once you are done with the exercises.